In [1]:

!pip -q install pymupdf python-pptx pillow transformers sentence-transformers scikit-learn tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 16.1 MB/s eta 0:00:00


In [2]:
import os, io, json, csv
from tqdm import tqdm
import fitz  # PyMuPDF
from pptx import Presentation
from PIL import Image

from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# summarizer: BART
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device_map="auto")

# embeddings: MiniLM
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# OCR engine + German language pack
!sudo apt-get -qq update && sudo apt-get -qq install -y tesseract-ocr tesseract-ocr-deu
!pip -q install pytesseract


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tesseract-ocr-deu.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-deu_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-deu (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-deu (1:4.00~git30-7274cfa-1.1) ...


In [4]:
import io, re
import pytesseract
from PIL import Image
import numpy as np

OCR_LANG = "eng+deu"      # supports English + German
OCR_PSM = "6"             # treat as a block of text
OCR_OEM = "3"             # default LSTM engine
OCR_CONFIG = f"--oem {OCR_OEM} --psm {OCR_PSM}"

def _clean_text(s: str) -> str:
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

def _ocr_pil(img: Image.Image) -> str:
    return _clean_text(pytesseract.image_to_string(img, lang=OCR_LANG, config=OCR_CONFIG))

def _pdf_page_to_image(page, dpi=300) -> Image.Image:
    pix = page.get_pixmap(dpi=dpi)
    return Image.open(io.BytesIO(pix.tobytes("png"))).convert("RGB")

def extract_pdf_text(path, ocr=True, min_chars_no_ocr=25, dpi=300):
    import fitz
    doc = fitz.open(path)
    slides, ocr_count = [], 0
    for i, page in enumerate(doc):
        raw = page.get_text().strip()
        used_ocr = False
        image_only = False
        if (not raw or len(raw) < min_chars_no_ocr) and ocr:
            img = _pdf_page_to_image(page, dpi=dpi)
            raw = _ocr_pil(img)
            used_ocr = True
            image_only = True
            ocr_count += 1
        slides.append({
            "index": i,
            "text": _clean_text(raw),
            "source": "pdf",
            "ocr_used": used_ocr,
            "image_only": image_only
        })
    print(f"OCR used on {ocr_count}/{len(slides)} PDF pages.")
    return slides

def extract_pptx_text(path, ocr=True):
    from pptx import Presentation
    prs = Presentation(path)
    slides, ocr_count = [], 0
    for i, s in enumerate(prs.slides):
        texts, images_ocr = [], []
        has_picture = False
        for shape in s.shapes:
            if hasattr(shape, "text") and shape.text:
                texts.append(shape.text)
            # 13 == picture
            if getattr(shape, "shape_type", None) == 13:
                has_picture = True
                if ocr:
                    try:
                        img = Image.open(io.BytesIO(shape.image.blob)).convert("RGB")
                        images_ocr.append(_ocr_pil(img))
                    except Exception:
                        pass
        raw = _clean_text("\n".join(t for t in texts if t))
        used_ocr = False
        image_only = False
        if ocr and (not raw or len(raw) < 25) and has_picture:
            ocr_txt = _clean_text("\n".join(t for t in images_ocr if t))
            if ocr_txt:
                raw = (raw + "\n" + ocr_txt).strip() if raw else ocr_txt
                used_ocr = True
                image_only = (len(_clean_text("\n".join(texts))) < 5)
                ocr_count += 1
        slides.append({
            "index": i,
            "text": raw,
            "source": "pptx",
            "ocr_used": used_ocr,
            "image_only": image_only
        })
    print(f"OCR used on {ocr_count}/{len(slides)} PPTX slides.")
    return slides


In [5]:
from google.colab import files
up = files.upload()  # choose your PDF or PPTX
file_path = list(up.keys())[0]
print(" Uploaded:", file_path)


Saving 1.MLISP_Introduction.pdf to 1.MLISP_Introduction.pdf
 Uploaded: 1.MLISP_Introduction.pdf


In [6]:
ext = extract_pdf_text if file_path.lower().endswith(".pdf") else extract_pptx_text
slides = ext(file_path, ocr=True)
len(slides), sum(s["ocr_used"] for s in slides), sum(s["image_only"] for s in slides)


OCR used on 0/38 PDF pages.


(38, 0, 0)

In [7]:
# Summarizer (multilingual: DE/EN)
from transformers import pipeline
try:
    summarizer
except NameError:
    summarizer = pipeline(
        "summarization",
        model="csebuetnlp/mT5_multilingual_XLSum",
        device_map="auto"
    )

# Embeddings (multilingual MiniLM)
from sentence_transformers import SentenceTransformer
try:
    embedder
except NameError:
    embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


In [8]:
from tqdm import tqdm

def summarize_text(txt):
    txt = (txt or "").strip()
    if not txt or len(txt.split()) < 25:
        return txt  # keep as-is for very short slides
    out = summarizer(txt, max_length=160, min_length=60, do_sample=False)[0]["summary_text"]
    return out.strip()

slide_summaries = []
for s in tqdm(slides):
    summary = summarize_text(s["text"])
    slide_summaries.append({
        "index": s["index"],
        "text": s["text"],
        "summary": summary,
        "ocr_used": s.get("ocr_used", False),
        "image_only": s.get("image_only", False),
        "source": s.get("source", "pdf")
    })

# quick peek
slide_summaries[0]


100%|██████████| 38/38 [08:00<00:00, 12.63s/it]


{'index': 0,
 'text': 'Prof. Dr. Vasileios Belagiannis\nChair of Multimedia Communications and Signal Processing\nMachine Learning in Signal Processing\nWinter Semester 2025/26\n1. Introduction\n14.10.2025',
 'summary': 'Prof. Dr. Vasileios Belagiannis\nChair of Multimedia Communications and Signal Processing\nMachine Learning in Signal Processing\nWinter Semester 2025/26\n1. Introduction\n14.10.2025',
 'ocr_used': False,
 'image_only': False,
 'source': 'pdf'}

In [26]:

import os, re, string, json, math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1) build source text from slides (preferred) or slide_summaries.json
try:
    _ = slides[0]
    pages = [s.get("text","") for s in slides if s.get("text")]
except Exception:
    with open("outputs/slide_summaries.json","r",encoding="utf-8") as f:
        _ss = json.load(f)
    pages = [s.get("text","") for s in _ss if s.get("text")]

doc = "\n".join(pages)

# 2) sentence splitting (DE/EN friendly)
sentences = re.split(r'(?<=[.!?])\s+', doc)
sentences = [s.strip() for s in sentences if len(s.strip().split()) >= 6]

# 3) TF-IDF ranking
stop = """
der die das ein eine und oder für von mit ohne zu auf an im in aus ist sind war waren sein auch sowie
the a an and or for from with without into in on of to is are was were be been being this that these those it its by as at if then else when while not
data using use based results method methods model models paper figure table slide page
""".split()

vectorizer = TfidfVectorizer(stop_words=stop, max_df=0.9, min_df=2, ngram_range=(1,2))
X = vectorizer.fit_transform(sentences)
scores = X.sum(axis=1).A.ravel()

# 4) MMR selection for diversity
def mmr_select(sentences, scores, X, max_words=260, lambda_div=0.7):
    chosen = []
    chosen_idx = []
    remaining = set(range(len(sentences)))
    current_len = 0
    sim = cosine_similarity(X, X)
    # pick best first
    i0 = int(scores.argmax())
    chosen.append(sentences[i0]); chosen_idx.append(i0)
    remaining.remove(i0); current_len += len(sentences[i0].split())
    # keep adding while under word budget
    while remaining and current_len < max_words:
        best_i, best_val = None, -1e9
        for i in remaining:
            relevance = scores[i]
            diversity = 0.0 if not chosen_idx else max(sim[i, chosen_idx])
            val = lambda_div*relevance - (1-lambda_div)*diversity
            if val > best_val:
                best_val, best_i = val, i
        s = sentences[best_i]
        if current_len + len(s.split()) > max_words + 40:  # small slack
            remaining.remove(best_i); continue
        chosen.append(s); chosen_idx.append(best_i); remaining.remove(best_i)
        current_len += len(s.split())
    # restore original document order
    ord_idx = sorted(chosen_idx)
    return " ".join(sentences[i] for i in ord_idx)

summary = mmr_select(sentences, scores, X, max_words=260, lambda_div=0.7)

# 5) light cleanup
summary = re.sub(r'\s+', ' ', summary).strip()

# 6) save
os.makedirs("outputs", exist_ok=True)
with open("outputs/deck_summary.txt","w",encoding="utf-8") as f:
    f.write(summary)

print("Extractive deck summary written to outputs/deck_summary.txt")
print(summary[:500])



Extractive deck summary written to outputs/deck_summary.txt
Machine Learning in Signal Processing Online reference: https://machinelearningmastery.com/analytical-vs-numerical-solutions-in-machine-learning/ By mohamed_hassan https://pixabay.com/de/vectors/machine-learning-bücher- algorithmus-6079971/ Chair of Multimedia Communications and Signal Processing Page 11 ***Not for sharing (Friedrich-Alexander-Universität Erlangen-Nürnberg)*** Machine Learning Milestones (Timeline) Machine Learning in Signal Processing 1805 • Least Squares 1812 •Bayes' Theorem 1


In [10]:
import numpy as np
from sentence_transformers import SentenceTransformer

try:
    embedder
except NameError:
    embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

corpus = [f"Slide {s['index']}: {s['summary'] or s['text']}" for s in slide_summaries]
embs = embedder.encode(corpus, convert_to_numpy=True, normalize_embeddings=True)

def search(query, top_k=5):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    sims = (embs @ q.T).ravel()
    top_idx = sims.argsort()[::-1][:top_k]
    return [
        {
            "slide_index": slide_summaries[i]["index"],
            "score": float(sims[i]),
            "preview": (slide_summaries[i]["summary"] or slide_summaries[i]["text"])[:240]
        }
        for i in top_idx
    ]

search("Markow-Eigenschaft")  # or any query


[{'slide_index': 15,
  'score': 0.5227386355400085,
  'preview': 'Markov Chain (1913) is a stochastic/random process that is memory-less. A Markov chains is a system of states with transition probabilities between the states. Applications: finance, genetics, computer vision, machine learning.Online Demo: '},
 {'slide_index': 27,
  'score': 0.39752787351608276,
  'preview': 'Reinforcement Learning (1989) is modelled as a Markov decision. The goal is to learn a policy to maximize the expected cumulative reward. Oleg Klimov, Chair of Multimedia Communications and Signal Processing at the University of Erlangen-Nü'},
 {'slide_index': 37,
  'score': 0.25981271266937256,
  'preview': 'Chair of Multimedia Communications and Signal Processing\nPage 38\n***Not for sharing (Friedrich-Alexander-Universität Erlangen-Nürnberg)***\nNext Lecture\nBasics & Terminology.\nMachine Learning in Signal Processing'},
 {'slide_index': 26,
  'score': 0.2542617917060852,
  'preview': 'Reinforcement Learning (19

In [11]:
import pandas as pd
df = pd.DataFrame([
    {"index": s["index"], "ocr": s["ocr_used"], "image_only": s["image_only"],
     "preview": s["text"][:120].replace("\n"," ")}
    for s in slides
])
df[df["ocr"] | df["image_only"]].head(10)  # shows any OCR'd slides


,index,ocr,image_only,preview


In [13]:
import re, os
from collections import Counter, defaultdict

def top_keywords(text, k=3):
    words = re.findall(r"[A-Za-zÄÖÜäöüß][A-Za-zÄÖÜäöüß\-]{2,}", text or "")
    stop = set("""
der die das ein eine und oder für von mit ohne zu auf an im in aus ist sind war waren sein auch sowie
the a an and or for from with without into in on of to is are was were be been being this that these those it its by as at if then else when while not
""".split())
    words = [w.lower() for w in words if w.lower() not in stop]
    return [w for w,_ in Counter(words).most_common(k)]

kw_to_slides = defaultdict(list)
for s in slide_summaries:
    base = s["summary"] or s["text"]
    for kw in top_keywords(base, k=3):
        kw_to_slides[kw].append(s["index"])

# keep top ~12 keywords
top_kws = [kw for kw,_ in Counter(kw_to_slides.keys()).most_common()]
top_kws = list(kw_to_slides.keys())[:12]

title = os.path.basename(file_path)
lines = ["mindmap", f"  root(({title}))"]
for kw in top_kws:
    lines.append(f"    {kw}({kw})")
    for idx in sorted(kw_to_slides[kw])[:6]:
        lines.append(f"      s{idx}([Slide {idx}])")

os.makedirs("outputs", exist_ok=True)
with open("outputs/mindmap.mmd", "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

print(" mindmap.mmd written to outputs/. Paste it into https://mermaid.live to render.")


 mindmap.mmd written to outputs/. Paste it into https://mermaid.live to render.


In [15]:
import csv, re

def split_points(text):
    bullets = re.findall(r"(?:^|\n)[\-\•\*]\s*(.+)", text or "")
    if bullets:
        return [b.strip() for b in bullets if len(b.strip()) > 3][:4]
    sents = re.split(r"(?<=[\.\!\?])\s+", text or "")
    return [s for s in sents if len(s.split()) >= 6][:3]

def detect_de(text: str) -> bool:
    t = (text or "").lower()
    if any(c in t for c in "äöüÄÖÜß"): return True
    de_hits = sum(w in t for w in ["und","der","die","das","ist","nicht","ein","eine","oder","auch"])
    en_hits = sum(w in t for w in ["and","the","is","are","not","or","also","a","an"])
    return de_hits >= en_hits

rows = []
for s in slide_summaries:
    base = s["summary"] or s["text"]
    if not base:
        continue
    de = detect_de(base)
    items = split_points(base)
    if items:
        for j, it in enumerate(items, 1):
            q = (f"Nenne einen Kerngedanken (Folie {s['index']}, Punkt {j}):"
                 if de else
                 f"Name one key idea (slide {s['index']}, point {j}):")
            rows.append([q, it, f"ClassMind;{'DE' if de else 'EN'}"])
    else:
        q = (f"Was ist die Kernaussage von Folie {s['index']}?"
             if de else
             f"What is the main idea of slide {s['index']}?")
        rows.append([q, base, f"ClassMind;{'DE' if de else 'EN'}"])

os.makedirs("outputs", exist_ok=True)
with open("outputs/flashcards.csv", "w", encoding="utf-8", newline="") as f:
    w = csv.writer(f); w.writerow(["Question","Answer","Tags"]); w.writerows(rows)

print(f"flashcards.csv written to outputs/ — total cards: {len(rows)}")


flashcards.csv written to outputs/ — total cards: 99


In [16]:
from transformers import pipeline

try:
    summarizer
except NameError:
    summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum", device=-1)

def chunk_by_tokens(text, tokenizer, max_tokens=480):
    enc = tokenizer(text, return_attention_mask=False, truncation=False)
    ids = enc["input_ids"]
    return [tokenizer.decode(ids[i:i+max_tokens], skip_special_tokens=True)
            for i in range(0, len(ids), max_tokens)]

def summarize_long_text(text, target_max=260, target_min=160, max_input_tokens=480):
    tok = summarizer.tokenizer
    if not text.strip():
        return ""
    parts = chunk_by_tokens(text, tok, max_tokens=max_input_tokens)
    partial = [summarizer(p, max_length=160, min_length=60, do_sample=False, truncation=False)[0]["summary_text"] for p in parts]
    merged = "\n".join(partial)
    final = summarizer(merged, max_length=target_max, min_length=target_min, do_sample=False, truncation=False)[0]["summary_text"]
    return final.strip()

full_text = "\n\n".join(s["text"] for s in slides if s["text"])
deck_summary = summarize_long_text(full_text)

with open("outputs/deck_summary.txt", "w", encoding="utf-8") as f:
    f.write(deck_summary)

print(" deck_summary.txt written to outputs/")


 deck_summary.txt written to outputs/


In [17]:
# Build (or reuse) embeddings over slide texts/summaries
slide_texts = [(s["index"], (s["summary"] or s["text"])) for s in slide_summaries]
corpus = [f"Slide {idx}: {txt}" for idx, txt in slide_texts]
embs = embedder.encode(corpus, convert_to_numpy=True, normalize_embeddings=True)

def retrieve(query, k=4):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    sims = (embs @ q.T).ravel()
    top = sims.argsort()[::-1][:k]
    ctx = []
    for i in top:
        idx, txt = slide_texts[i]
        ctx.append(f"[Slide {idx}] {txt}")
    return ctx, sims[top].tolist()

def answer(query):
    ctx, scores = retrieve(query, k=4)
    prompt = (
        "Beantworte die Frage kurz und präzise, basierend NUR auf dem Kontext.\n"
        "Wenn die Antwort nicht im Kontext steht, sage: 'Nicht im Material enthalten.'\n\n"
        "KONTEXT:\n" + "\n\n".join(ctx) + "\n\nFRAGE:\n" + query + "\n\nANTWORT:"
    )
    out = summarizer(prompt, max_length=120, min_length=30, do_sample=False)[0]["summary_text"]
    return out.strip(), ctx, scores

ans, ctx, scores = answer("Was ist die Markow-Eigenschaft?")
print(ans)


LEARNING: These slides contains a summary of the Markov Chain process - which makes it easier to understand.


In [19]:
# Fix a broken .ipynb so GitHub renders it (adds outputs: [], sets execution_count, strips widgets)
!pip -q install nbformat==5.10.4
import nbformat as nbf
from nbformat.validator import validate
from google.colab import files

print("Pick the problematic .ipynb file from your computer:")
up = files.upload()
name = list(up.keys())[0]
raw = up[name].decode('utf-8')

nb = nbf.reads(raw, as_version=4)

# repo-level metadata (safe defaults)
nb.metadata.pop('widgets', None)
nb.metadata.setdefault('kernelspec', {'name': 'python3', 'display_name': 'Python 3'})
nb.metadata.setdefault('language_info', {'name': 'python'})

# ensure every code cell has required fields
for c in nb.cells:
    # remove stray widget metadata
    if isinstance(c.get('metadata', {}), dict):
        c.metadata.pop('widgets', None)

    # source must be a string
    if isinstance(c.get('source'), list):
        c.source = ''.join(c.source)

    if c.get('cell_type') == 'code':
        if c.get('outputs') is None:
            c['outputs'] = []
        if 'outputs' not in c:
            c['outputs'] = []
        if 'execution_count' not in c:
            c['execution_count'] = None

# set nbformat headers
nb.nbformat = 4
nb.nbformat_minor = 5

# validate schema
validate(nb)

clean_name = name.rsplit('.ipynb', 1)[0] + "_clean.ipynb"
with open(clean_name, "w", encoding="utf-8") as f:
    nbf.write(nb, f)

print(" Cleaned notebook created:", clean_name)
files.download(clean_name)


🔼 Pick the problematic .ipynb file from your computer:


Saving Untitled29.ipynb to Untitled29 (1).ipynb
✅ Cleaned notebook created: Untitled29 (1)_clean.ipynb


/tmp/ipython-input-2641448564.py:42: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
import json, os

os.makedirs("outputs", exist_ok=True)
with open("outputs/slide_summaries.json", "w", encoding="utf-8") as f:
    json.dump(slide_summaries, f, ensure_ascii=False, indent=2)

print(" slide_summaries.json saved to outputs/")

from google.colab import files
for p in ["outputs/deck_summary.txt",
          "outputs/slide_summaries.json",
          "outputs/mindmap.mmd",
          "outputs/flashcards.csv"]:
    files.download(p)


✅ slide_summaries.json saved to outputs/


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
import re, json, os

path = "outputs/deck_summary.txt"
with open(path, "r", encoding="utf-8") as f:
    s = f.read()

# remove URLs and image credits / licenses
s = re.sub(r'https?://\S+', ' ', s)
s = re.sub(r'\b(CC BY|Pixabay|Wikimedia|commons\.wikimedia)\b.*?(?=[A-Z]|\Z)', ' ', s, flags=re.I|re.S)
s = re.sub(r'\bBy [A-Z][^,]{0,60},?\s?', ' ', s)

# remove watermark/headers like “Chair of … Page 12 ***Not for sharing***”
s = re.sub(r'Chair of .*?Universität.*?\*{3}.*?\*{3}', ' ', s, flags=re.I)
s = re.sub(r'\bPage\s*\d+\b', ' ', s, flags=re.I)

# collapse bullets/stray glyphs and extra whitespace
s = re.sub(r'•', ' - ', s)
s = re.sub(r'\s{2,}', ' ', s).strip()

# de-duplicate repeated sentences
sentences = [t.strip() for t in re.split(r'(?<=[.!?])\s+', s) if t.strip()]
seen, uniq = set(), []
for t in sentences:
    key = re.sub(r'\W+', '', t.lower())
    if key not in seen:
        seen.add(key)
        uniq.append(t)
cleaned = ' '.join(uniq)

with open(path, "w", encoding="utf-8") as f:
    f.write(cleaned)

print("Cleaned deck_summary.txt saved.")
print(cleaned[:500])


Cleaned deck_summary.txt saved.
Machine Learning in Signal Processing Online reference: By mohamed_hassan algorithmus-6079971/ Not for sharing (Friedrich-Alexander-Universität Erlangen-Nürnberg)*** Least Squares ( Adrien-Marie Legendre, 1805) - Find the best-fitting curve for a set of points, e.g. ications and Signal Processing ***Not for sharing (Friedrich-Alexander-Universität Erlangen-Nürnberg)*** Turing Test and (Cont.) - Originally called: imitation game. ications and Signal Processing ***Not for sharing (Friedrich-Alexan


In [29]:
from google.colab import files
files.download("outputs/deck_summary.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>